In [2]:
import pandas as pd
import os
import glob
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category = FutureWarning)
from tqdm import tqdm
import sys
import re

sys.path.insert(0, '../../../src/utils')
from import_data import *

def summ_followers2(df):
    if 'like_count' in df.columns:
        df = df.rename(columns={'like_count':'total_likes'})
    else:
        df = df.rename(columns={'public_metrics.like_count':'total_likes'}) 
    df['t_verifiability_smi'] = df['verifiability'] * df['interaction_smi']
    df['t_verifiability_ac'] = df['verifiability'] * df['interaction_ac']
    df['t_eng_smi'] = df['eng'] * df['interaction_smi']
    df['t_eng_ac'] = df['eng'] * df['interaction_ac']
    df['t_non_ver_smi'] = df['non_ver'] * df['interaction_smi']
    df['t_non_ver_ac'] = df['non_ver'] * df['interaction_ac']
    df['t_true_smi'] = df['true'] * df['interaction_smi']
    df['t_true_ac'] = df['true'] * df['interaction_ac']
    df['t_fake_smi'] = df['fake'] * df['interaction_smi']
    df['t_fake_ac'] = df['fake'] * df['interaction_ac']

    
    cols = (['interaction_smi', 'interaction_ac'] + 
            [col for col in df.columns if 't_' in col])
    df_agg = df[['author_id'] + 
        cols].groupby(['author_id']).sum().reset_index()
    
    return df_agg

smi_ke = pd.read_excel("~/Dropbox/Bolivia_Project/social-media-influencers-africa/data/02-randomize/KE/03-assignment/output/RandomizedTwitterSampleKE.xlsx")
smi_ke_1 = pd.read_excel("~/Dropbox/Bolivia_Project/social-media-influencers-af/data/02-randomize/KE/03-assignment/output/RandomizedTwitterSampleKE.xlsx")
smi_ke_2 = pd.read_excel("~/Dropbox/Bolivia_Project/social-media-influencers-af/data/02-randomize/KE/03-assignment/output/RandomizedTwitterSampleKE_batch2.xlsx")

smi_sa = pd.read_excel("~/Dropbox/Bolivia_Project/social-media-influencers-africa/data/02-randomize/SA/03-assignment/output/RandomizedTwitterSampleSA.xlsx")
smi_sa_1 = pd.read_excel("~/Dropbox/Bolivia_Project/social-media-influencers-af/data/02-randomize/SA/03-assignment/output/RandomizedTwitterSampleSA.xlsx")
smi_sa_2 = pd.read_excel("~/Dropbox/Bolivia_Project/social-media-influencers-af/data/02-randomize/SA/03-assignment/output/RandomizedTwitterSampleSA_batch2.xlsx")

smis = pd.concat([smi_ke,smi_ke_1,smi_ke_2,smi_sa,smi_sa_1,smi_sa_2], ignore_index=True)
smis = smis[smis['treatment']==1]
smis_list = list(smis.username)

#This I do recommend running it first for one country, restart kernel, and run for the other country:

In [3]:
country = 'SA'

base1 = pd.read_parquet(f'../../../data/04-analysis/{country}/treatment_info/information_batch1.parquet')

df_final = get_baseline_data_b1(country, type_data = 'predicted', base_path = '../../../')
df_final['fake'] = np.where((df_final['verifiability'] == 1) & (df_final['true'] == 0), 1, 0)
df_final['eng'] = np.where((df_final['lang'] == 'en'), 1, 0)
df_final['fake'] = np.where((df_final['verifiability'] == 0), np.nan, df_final['fake'])
df_final['non_ver'] = np.where((df_final['verifiability'] == 0) & (~df_final['verifiability'].isnull()),
                                   1, 0)
# Extraer el nombre de usuario de los retweets
df_final['reposted'] = df_final['text'].str.extract(r'^RT @(\w+):')
# Completar los valores nulos con la columna handle
df_final['reposted'] = df_final['reposted'].fillna(df_final['handle'])
df_final['interaction_ac'] = (df_final['reposted'] == 'AfricaCheck').astype(int)
df_final['interaction_smi'] = (df_final['reposted'].isin(smis_list)).astype(int)
df_final = df_final.drop(['reposted'], axis = 1)
df_no_rt = df_final[(df_final['text'].str.contains('RT @', 
                    case=True, regex=False))].reset_index(drop=True)

df_no_rt_agg = summ_followers2(df_no_rt).reset_index(drop=True)

cols = (['interaction_smi', 'interaction_ac'] + 
            [col for col in df_no_rt_agg.columns if 't_' in col])

df_no_rt_agg.rename(columns = 
        {col: col + '_base' for col in df_no_rt_agg.columns if col in cols}, 
        inplace=True)

df_no_rt_agg.rename(columns = { 
                       'author_id':'follower_id'}, inplace = True)


base1 = base1.merge(df_no_rt_agg, on=['follower_id'], 
                    how='left')

cols_base = [col for col in base1.columns if '_base' in col]
for x in cols_base:
    base1[x] = np.where(base1[x].isnull(), 0, base1[x])
    
base1.to_parquet(f'../../../data/04-analysis/{country}/baseline/baseline_batch1_smi_ac.parquet')

100%|███████████████████████████████████████████| 74/74 [27:11<00:00, 22.04s/it]


In [3]:
country = 'KE'

base1 = pd.read_parquet(f'../../../data/04-analysis/{country}/treatment_info/information_batch1.parquet')

df_final = get_baseline_data_b1(country, type_data = 'predicted', base_path = '../../../')
df_final['fake'] = np.where((df_final['verifiability'] == 1) & (df_final['true'] == 0), 1, 0)
df_final['eng'] = np.where((df_final['lang'] == 'en'), 1, 0)
df_final['fake'] = np.where((df_final['verifiability'] == 0), np.nan, df_final['fake'])
df_final['non_ver'] = np.where((df_final['verifiability'] == 0) & (~df_final['verifiability'].isnull()),
                                   1, 0)
# Extraer el nombre de usuario de los retweets
df_final['reposted'] = df_final['text'].str.extract(r'^RT @(\w+):')
# Completar los valores nulos con la columna handle
df_final['reposted'] = df_final['reposted'].fillna(df_final['handle'])
df_final['interaction_ac'] = (df_final['reposted'] == 'AfricaCheck').astype(int)
df_final['interaction_smi'] = (df_final['reposted'].isin(smis_list)).astype(int)
df_final = df_final.drop(['reposted'], axis = 1)
df_no_rt = df_final[(df_final['text'].str.contains('RT @', 
                    case=True, regex=False))].reset_index(drop=True)

df_no_rt_agg = summ_followers2(df_no_rt).reset_index(drop=True)

cols = (['interaction_smi', 'interaction_ac'] + 
            [col for col in df_no_rt_agg.columns if 't_' in col])

df_no_rt_agg.rename(columns = 
        {col: col + '_base' for col in df_no_rt_agg.columns if col in cols}, 
        inplace=True)

df_no_rt_agg.rename(columns = { 
                       'author_id':'follower_id'}, inplace = True)


base1 = base1.merge(df_no_rt_agg, on=['follower_id'], 
                    how='left')

cols_base = [col for col in base1.columns if '_base' in col]
for x in cols_base:
    base1[x] = np.where(base1[x].isnull(), 0, base1[x])
    
base1.to_parquet(f'../../../data/04-analysis/{country}/baseline/baseline_batch1_smi_ac.parquet')

100%|███████████████████████████████████████████| 84/84 [57:58<00:00, 41.41s/it]


In [4]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30182655 entries, 0 to 30182654
Data columns (total 23 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   id                          object 
 1   handle                      object 
 2   author_id                   object 
 3   created_at                  object 
 4   text                        object 
 5   lang                        object 
 6   referenced_tweets           object 
 7   public_metrics.like_count   int64  
 8   public_metrics.quote_count  int64  
 9   public_metrics.reply_count  int64  
 10  total_shares                int64  
 11  total_reactions             int64  
 12  total_comments              int64  
 13  has_url                     int64  
 14  has_image                   int64  
 15  has_text                    int64  
 16  verifiability               float64
 17  true                        float64
 18  fake                        float64
 19  eng                